**Задание 1.** Реализовать алгоритм блочного шифрования в режиме сцепления блоков шифротекста на основе скиталы 

In [133]:
import numpy as np
import time

In [3]:
import secrets
import string
# Функция генерации символьной строки заданного размера
def generate_random_string_rus(length):
    rus_letters_and_digits = "АБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯ" + " " + "1234567890" # исходная строка из букв русского алфавита, цифр и пробела
    rand_string = ''.join(secrets.choice(
        rus_letters_and_digits) for i in range(length)) # функция secrets.choice() возвращает случайно выбранный элемент из последовательности
                                                    # таким образом получаем строку заданной длины
    return rand_string

In [4]:
import secrets
import string

def generate_random_string_eng(length):
    eng_letters = "ABCDE" 
    rand_string = ''.join(secrets.choice(
    eng_letters) for i in range(length)) # функция secrets.choice() возвращает случайно выбранный элемент из последовательности
                                                    # таким образом получаем строку заданной длины
    return rand_string

In [5]:
# Функция шифрования
def block_encryption(source_txt, rnd_str, n, m):
  start = time.time() # время начала отсчета
  blocks_txt = []
  for x in range (0, len(source_txt), n*m): # цикл для генерации списка с нарезками текста на блоки равной длинны 
    if (x+n*m) > len(source_txt): 
      source_txt = source_txt.ljust(x+n*m, ' ') # добавляем недостающие пробелы в конце строки 
    blocks_txt.append(source_txt[x:x+n*m])
  print("Блоки исходного текста:", blocks_txt)
  resultXor = ""
  txt_encrypt = ""
  for block in blocks_txt:
    for i, ch in enumerate(block):
      resultXor += chr(ord(ch) ^ ord(rnd_str[i]))
    lst = np.array(list(resultXor))
    mass = lst.reshape(n, m)
    mass_tr = mass.transpose() # транспонирование матрицы
    print(mass_tr)
    block_encrypt = ""
    for row in mass_tr:
      for ch in row:
        block_encrypt += ch # получаем первый блок зашифрованного сообщения
    txt_encrypt += block_encrypt
    print(block_encrypt)
    break

  for i in range(1, len(blocks_txt)): # обход блоков исходного текста, начиная с 1ого индекса, т.к. блок с индексом 0 уже был зашифрован
    resultXor = ""
    for j in range(len(blocks_txt[i])):
      resultXor += chr(ord(blocks_txt[i][j]) ^ ord(block_encrypt[j]))
    lst = np.array(list(resultXor))
    mass = lst.reshape(n, m)
    mass_tr = mass.transpose() # транспонирование матрицы
    print(mass_tr)
    block_encrypt = ""
    for row in mass_tr:
      for ch in row:
        block_encrypt += ch # получаем следующий блок зашифрованного сообщения
    print(block_encrypt)
    txt_encrypt += block_encrypt
  end = time.time() # время конца отсчета
  print("Время выполнения шифрования:", end-start)
  return txt_encrypt

In [15]:
input_text = generate_random_string_eng(20)
print('Исходный текст:', input_text)
n = 3 # ключ Скиталла (диаметр палочки, кол-во строк в матрице)
m = 3  # кол-во столбцов
rnd_str = generate_random_string_eng(n*m)
print('Вектор инициализации:', rnd_str)
txt_encrypt = block_encryption(input_text, rnd_str, n, m)
print("Зашифрованный текст:", txt_encrypt)

Исходный текст: DEDECDCACEDAADBBAEAC
Вектор инициализации: BCAAAEABE
Блоки исходного текста: ['DEDECDCAC', 'EDAADBBAE', 'AC       ']
[['\x06' '\x04' '\x02']
 ['\x06' '\x02' '\x03']
 ['\x05' '\x01' '\x06']]

[['C' 'G' 'G']
 ['@' 'F' '@']
 ['C' 'A' 'C']]
CGG@F@CAC
[['\x02' '`' 'c']
 ['\x04' 'f' 'a']
 ['g' '`' 'c']]
`cfag`c
Время выполнения шифрования: 0.0009968280792236328
Зашифрованный текст: CGG@F@CAC`cfag`c


**Задание 2.** Реализовать обратную операцию расшифровки (см. рисунок 1): каждый зашифрованный блок сначала надо расшифровать по методу скиталы, затем сложить по XOR с предыдущим зашифрованным блоком (в начале с тем же вектором инициализации, что был использован при шифровании)

In [10]:
# Функция расшифровки
def block_decryption(txt_encrypt, rnd_str, n, m):
  start = time.time() # время начала отсчета  
  block_encrypt = [txt_encrypt[x:x+n*m] for x in range (0, len(txt_encrypt), n*m)] #генератор списка, использующий срезы для нарезки зашифрованного текста на блоки равной длинны
  for block in block_encrypt:
    lst = np.array(list(block))
    print(lst)
    mass = lst.reshape(m, n)
    mass_tr = mass.transpose() # транспонирование матрицы
    print(mass_tr)
    result_Xor = ""
    for row in mass_tr:
      result_Xor += ''.join(map(str, row)) # из транспонированной к первоначальному виду матрицы получаю строку, которая изначально была получена операцией XOR
    print(result_Xor)
    block_decrypt = ""
    txt_decrypt = ""
    for i, ch in enumerate(result_Xor):
      block_decrypt += chr(ord(ch) ^ ord(rnd_str[i])) # получили первый блок исходного сообщения
    txt_decrypt += block_decrypt
    print(block_decrypt)
    break

  for i in range(1, len(block_encrypt)):
    lst = np.array(list(block_encrypt[i]))
    print(lst)
    mass = lst.reshape(m, n)
    mass_tr = mass.transpose() # транспонирование матрицы
    print(mass_tr)
    result_Xor = ""
    for row in mass_tr:
      result_Xor += ''.join(map(str, row)) # из транспонированной к первоначальному виду матрицы получаю строку, которая изначально была получена операцией XOR
    print(result_Xor)
    block_decrypt = ""
    for j, ch in enumerate(result_Xor):
      block_decrypt += chr(ord(ch) ^ ord(block_encrypt[i-1][j])) # получили следующий блок исходного сообщения
    txt_decrypt += block_decrypt
    print(block_decrypt)
  end = time.time() # время конца отсчета
  print("Время выполнения расшифровки:", end-start)
  return txt_decrypt

In [16]:
txt_decrypt = block_decryption(txt_encrypt, rnd_str, n, m)
print("Расшифрованный текст:", txt_decrypt)

['\x06' '\x04' '\x02' '\x06' '\x02' '\x03' '\x05' '\x01' '\x06']
[['\x06' '\x06' '\x05']
 ['\x04' '\x02' '\x01']
 ['\x02' '\x03' '\x06']]

DEDECDCAC
['C' 'G' 'G' '@' 'F' '@' 'C' 'A' 'C']
[['C' '@' 'C']
 ['G' 'F' 'A']
 ['G' '@' 'C']]
C@CGFAG@C
EDAADBBAE
['\x02' '`' 'c' '\x04' 'f' 'a' 'g' '`' 'c']
[['\x02' '\x04' 'g']
 ['`' 'f' '`']
 ['c' 'a' 'c']]
g`f`cac
AC       
Время выполнения расшифровки: 0.0013957023620605469
Расшифрованный текст: DEDECDCACEDAADBBAEAC       


**Задание 3.** Модифицировать код так, чтобы он работал теперь с содержимым текстовых файлов. Один содержит исходный текст, второй -- зашифрованный, третий -- результат расшифровки. 

In [17]:
def ReadingFile (path): # функция чтения файла
  #txt = []
  txt = ''
  f = None
  try:
      f = open(path, 'r', encoding="utf-8") #Открыть файл для чтения
      for ln in f: #Обход строк файла и добавление в список
          #txt.extend(ln) # Добавить к списку
          txt += ln # символ окончания строки заменяем на пробел
  except Exception as e:
      print("Ошибка при работе с файлом:", e)
  finally:
      if f:  # Если файл не открылся, значит 'f' == None и закрывать его не нужно
          f.close()
  return txt

In [18]:
def WritingFile (path, w_txt): # функция записи в файл
  f = None
  try:
      f = open(path, 'w', encoding="utf-8") #Открыть файл для записи
      f.write(w_txt) # метод write() - запись в 
      print("Текст успешно записан в файл")
  except Exception as e:
      print("Ошибка при работе с файлом:", e)
  finally:
      if f:  # Если файл не открылся, значит 'fh' == None и закрывать его не нужно
          f.close()

In [19]:
input_text = ReadingFile('source_txt.txt')
print('Исходный текст:', input_text)
print("Длинна текста:", len(input_text))
n = 4
m = 3
rnd_str = generate_random_string_eng(n*m)
print('Вектор инициализации:', rnd_str)
txt_encrypt = block_encryption(input_text, rnd_str, n, m)
WritingFile('encoded_txt.txt', txt_encrypt) # запись зашифрованного текста в отдельный файл
print("Зашифрованный текст:", ReadingFile('encoded_txt.txt'))

Исходный текст: Привет мир! Доброе утро мир! Привет мир! Доброе утро мир! Привет мир! Доброе утро мир! Привет мир! Доброе утро мир! Привет мир! Доброе утро мир! Привет мир! Доброе утро мир! Привет мир! Доброе утро мир!
Длинна текста: 202
Вектор инициализации: CBDACAADDBAD
Блоки исходного текста: ['Привет мир! ', 'Доброе утро ', 'мир! Привет ', 'мир! Доброе ', 'утро мир! Пр', 'ивет мир! До', 'брое утро ми', 'р! Привет ми', 'р! Доброе ут', 'ро мир! Прив', 'ет мир! Добр', 'ое утро мир!', ' Привет мир!', ' Доброе утро', ' мир! Привет', ' мир! Доброе', ' утро мир!  ']
[['ќ' 'ѳ' 'a' 'Ђ']
 ['Ђ' 'Ѷ' 'Ѹ' '`']
 ['Ѽ' 'Ѓ' 'Ѽ' 'd']]
ќѳaЂЂѶѸ`ѼЃѼd
[['H' 'B' 'ј' 'C']
 ['M' '<' 'У' 'B']
 ['ѐ' 'C' '>' 'D']]
HBјCM<УBѐC>D
[['Ѵ' 'b' 'c' 'Ѷ']
 ['Ѻ' 'm' 'Ѻ' 'Ѽ']
 ['\x18' 'У' 'b' 'd']]
ѴbcѶѺmѺѼУbd
[['H' 'ї' 'D' '\x1d']
 ['њ' 'њ' 'M' 'ї']
 ['У' 'ѹ' 'ј' 'D']]
HїDњњMїУѹјD
[['Ћ' 'У' 'ѵ' 'љ']
 ['\x15' 'Ѻ' '\x17' 'G']
 ['Є' 'f' 'Ђ' 'Є']]
ЋУѵљѺGЄfЂЄ
[['3' '\x1b' 'Я' 'F']
 ['\x11' '5' 'Ї' '\x16']
 ['@' 'F' 'Х' ':

In [20]:
txt_decrypt = block_decryption(txt_encrypt, rnd_str, n, m)
WritingFile('decoded_txt.txt', txt_decrypt) # запись расшифрованного текста в отдельный файл
print("Расшифрованный текст:", ReadingFile('decoded_txt.txt'))

['ќ' 'ѳ' 'a' 'Ђ' 'Ђ' 'Ѷ' 'Ѹ' '`' 'Ѽ' 'Ѓ' 'Ѽ' 'd']
[['ќ' 'Ђ' 'Ѽ']
 ['ѳ' 'Ѷ' 'Ѓ']
 ['a' 'Ѹ' 'Ѽ']
 ['Ђ' '`' 'd']]
ќЂѼѳѶЃaѸѼЂ`d
Привет мир! 
['H' 'B' 'ј' 'C' 'M' '<' 'У' 'B' 'ѐ' 'C' '>' 'D']
[['H' 'M' 'ѐ']
 ['B' '<' 'C']
 ['ј' 'У' '>']
 ['C' 'B' 'D']]
HMѐB<CјУ>CBD
Доброе утро 
['Ѵ' 'b' 'c' 'Ѷ' 'Ѻ' 'm' 'Ѻ' 'Ѽ' '\x18' 'У' 'b' 'd']
[['Ѵ' 'Ѻ' '\x18']
 ['b' 'm' 'У']
 ['c' 'Ѻ' 'b']
 ['Ѷ' 'Ѽ' 'd']]
ѴѺbmУcѺbѶѼd
мир! Привет 
['H' 'ї' 'D' '\x1d' 'њ' 'њ' 'M' 'ї' 'У' 'ѹ' 'ј' 'D']
[['H' 'њ' 'У']
 ['ї' 'њ' 'ѹ']
 ['D' 'M' 'ј']
 ['\x1d' 'ї' 'D']]
HњУїњѹDMјїD
мир! Доброе 
['Ћ' 'У' 'ѵ' 'љ' '\x15' 'Ѻ' '\x17' 'G' 'Є' 'f' 'Ђ' 'Є']
[['Ћ' '\x15' 'Є']
 ['У' 'Ѻ' 'f']
 ['ѵ' '\x17' 'Ђ']
 ['љ' 'G' 'Є']]
ЋЄУѺfѵЂљGЄ
утро мир! Пр
['3' '\x1b' 'Я' 'F' '\x11' '5' 'Ї' '\x16' '@' 'F' 'Х' ':']
[['3' '\x11' '@']
 ['\x1b' '5' 'F']
 ['Я' 'Ї' 'Х']
 ['F' '\x16' ':']]
3@5FЯЇХF:
ивет мир! До
['Ђ' 'ѳ' 'E' 'f' 'ћ' '1' 'і' '\x19' '\x11' 'Ѷ' 'Ѿ' 'Ђ']
[['Ђ' 'ћ' '\x11']
 ['ѳ' '1' 'Ѷ']
 ['E' 'і' 'Ѿ']
 ['f' '\x19' 'Ђ']]
Ђћѳ1ѶEіѾfЂ


**Задание 4.** Зашифровать таким образом 4 разных файла (длиной не меньше 1000 символов) с текстом с разными ключами с ограничением 12<=n*m<=32.

In [70]:
input_text = ReadingFile('source_txt_1.txt')
print('Исходный текст:', input_text)
print("Длина текста:", len(input_text))
n = 4
m = 3
rnd_str = generate_random_string_eng(n*m)
print('Вектор инициализации:', rnd_str)
txt_encrypt = block_encryption(input_text, rnd_str, n, m)
WritingFile('encoded_txt_1.txt', txt_encrypt) # запись зашифрованного текста в отдельный файл
print("Зашифрованный текст:", ReadingFile('encoded_txt_1.txt'))

Исходный текст: Чаще всего искусственным интеллектом, или ИИ (Artificial Intelligence – AI), называют процесс создания машин, которые способны действовать таким образом, что будут восприниматься человеком как разумные. Это может быть повторение поведения человека или выполнение более простых задач, например, выживание в динамически меняющейся обстановке (поведение насекомых, животных и т.п.).
В период становления ИИ создание разумных систем казалось очень простой задачей, которая так и не была решена. В наше время цели создания ИИ стали намного практичнее. Соответственно искусственный интеллект в настоящее время толкуется, как свойство автоматических систем брать на себя отдельные функции интеллекта человека, например, выбирать и принимать оптимальные решения на основе ранее полученного опыта и рационального анализа внешних воздействий.
Как правило, интеллектом называется способность мозга решать (интеллектуальные) задачи путем приобретения, запоминания и целенаправленного преобразован

In [72]:
input_text = ReadingFile('source_txt_2.txt')
print('Исходный текст:', input_text)
print("Длина текста:", len(input_text))
n = 2
m = 7
rnd_str = generate_random_string_eng(n*m)
print('Вектор инициализации:', rnd_str)
txt_encrypt = block_encryption(input_text, rnd_str, n, m)
WritingFile('encoded_txt_2.txt', txt_encrypt) # запись зашифрованного текста в отдельный файл
print("Зашифрованный текст:", ReadingFile('encoded_txt_2.txt'))

Исходный текст: Среди других преимуществ перехода на процессно-ориентированную организацию деятельности можно выделить простоту проведения оптимизации как самих процессов, с точки зрения их организации, синхронизации, взаимной согласованности, так и ресурсов, потребляемых процессами, особенно это касается человеческих ресурсов.
Общепризнанно, что одним из важнейших факторов успеха предприятия является наличие на нем единой мощной "управленческой команды", владеющей ситуацией и работающей согласованно на достижение общей цели. Процессная организация управления в сочетании с другими методами позволяет сформировать такую команду, ориентированную на единые цели предприятия.
Таким образом, необходимость использования процессного подхода становится все более и более очевидной. Внедрение такого подхода требует осознанной реорганизации деятельности предприятия на основе принципов процессно-ориентированного управления. Подобная реорганизация требует определенной технологии перехода от существую

In [73]:
input_text = ReadingFile('source_txt_3.txt')
print('Исходный текст:', input_text)
print("Длина текста:", len(input_text))
n = 3
m = 5
rnd_str = generate_random_string_eng(n*m)
print('Вектор инициализации:', rnd_str)
txt_encrypt = block_encryption(input_text, rnd_str, n, m)
WritingFile('encoded_txt_3.txt', txt_encrypt) # запись зашифрованного текста в отдельный файл
print("Зашифрованный текст:", ReadingFile('encoded_txt_3.txt'))

Исходный текст: Физическая культура - часть общей культуры общества, одна из сфер социальной деятельности, направленная на поддержание здоровья, развитие физических способностей человека и использование их в соответствии с потребностями общественной практики.
Под физической культурой понимают совокупность всех присущих данному обществу целей, задач, средств, форм мероприятий, способствующих физическому развитию и совершенствованию людей. Сюда входят физическое воспитание, спорт и т.п.
Физическое воспитание - это педагогический процесс, который направлен на совершенствование форм и функций организма человека, формирование двигательных навыков, умений связанных с ними знаний, а также на воспитание физических качеств. Физическое воспитание связано с другими сторонами воспитания: нравственной, эстетической, производственной, трудовой.
Физическая культура студентов представляет собой неразрывную составную часть высшего образования. Она выступает качественной и результирующей мерой комплексн

In [121]:
input_text = ReadingFile('source_txt_4.txt')
print('Исходный текст:', input_text)
print("Длина текста:", len(input_text))
n = 4
m = 4
rnd_str = generate_random_string_eng(n*m)
print('Вектор инициализации:', rnd_str)
txt_encrypt = block_encryption(input_text, rnd_str, n, m)
WritingFile('encoded_txt_4.txt', txt_encrypt) # запись зашифрованного текста в отдельный файл
print("Зашифрованный текст:", ReadingFile('encoded_txt_4.txt'))

Исходный текст: Как бы не были организованы индексы в конкретной СУБД, их основное назначение состоит в обеспечении эффективного прямого доступа к кортежу отношения по ключу. Обычно индекс определяется для одного отношения, и ключом является значение атрибута (возможно, составного). Если ключом индекса является возможный ключ отношения, то индекс должен обладать свойством уникальности, т.е. не содержать дубликатов ключа. На практике ситуация выглядит обычно противоположно: при объявлении первичного ключа отношения автоматически заводится уникальный индекс, а единственным способом объявления возможного ключа, отличного от первичного, является явное создание уникального индекса. Это связано с тем, что для проверки сохранения свойства уникальности возможного ключа, так или иначе, требуется индексная поддержка.
Поскольку при выполнении многих операций языкового уровня требуется сортировка отношений в соответствии со значениями некоторых атрибутов, полезным свойством индекса является обеспе

**Задание 5.** С учетом этого ограничения перебрать возможные ключи n и m (при условии, что вы знаете, что размер блока должен быть кратен числу символов в зашифрованном сообщении) и возможные для каждого их них значения векторов инициализации c0 длиной n*m.

In [122]:
from math import log2
from collections import Counter
# Функция подсчета энтропии
def entropy (txt):
    dic = Counter(txt) # словарь с числом появлений каждого символа текста
    total = sum(dic.values()) # длину строки можно рассчитать как сумму всех частот символов
    
    return round(sum(val / total * log2(total / val) for val in dic.values()), 5)

In [123]:
from itertools import product
# Функция перебора символов в строке
def str_search (leng):
  start = time.time() # время начала отсчета
  eng_letters = "ABCDE"
  kol = 0
  for combos in product(eng_letters, repeat=leng): 
    S = ''.join(combos)
    kol += 1
    print(kol)
    #print(S)
  end = time.time() # время конца отсчета
  print("Время перебора вектора инициализации", end-start)


In [135]:
encoded_txt_1 = ReadingFile('encoded_txt_1.txt')
print("Зашифрованный текст:", encoded_txt_1)
input_txt_1 = ReadingFile('source_txt_1.txt')
print("Энтропия исходного текста:", entropy(input_txt_1))
print("Энтропия зашифрованного текста:", entropy(encoded_txt_1))

start = time.time() # время начала отсчета
n = 2
m = 2
while n <= 16:
  if n * m < 12:
    m += 1
    continue
  while n*m >= 12 and n*m <=36: # перебор ключей
    if len(encoded_txt_1)%(n*m) == 0:
      print("Возможный ключ:", n, m)  
      #print("Время перебора вектора инициализации:", str_searh(n*m))
    m += 1
  n += 1
  m = 2
end = time.time() # время конца отсчета
print("Время перебора ключей:", end-start)

Зашифрованный текст: ѣѴЂѽѲeѷbЊѰѰѽ"<EMN3ї;AЧ7AЏleЄѻЎЉѸЈѻѽ.ўРЯ<кєћјЏZщpВзьіжpUѴѻ?ѻиuХѐѕ⑧Щћ~YeвЅ⁐ХБmїмћ⑥MydњCPЫѰDЋoсЪѨ⁕уj'5jѣѐЭ#ЫѾ①ЩЙRцѴm‡єfKk+v?ќѬ6④ѳбСdї⁝Ёш\OУЎЁНеBЍ?<|Ѓ\⑬П<ЪuЇAPѺЀА0ь‭mрѢ.v<OР␝ѿ:їѠс"ѿ⁖0ОѳJz"Ш④BМѲЁ=IоиѮѼLѴЈэA⁜^Ј
юFѴБC=лШѲy⑤ъ{#тѽѶЈ⁜JЇJѪщ|=2ѨF②jдKѲ[wЇ‡Љчшє9ЈѹuОW=ѸI0wъ␑5^ѯЉЫзЁЇMW‧ѮRЗеѧ13?␛<ѰVЕ]Ѓѳ"Љ‮>tУП <␟УѤѿB7ж!ВЃAЃІьЬ‪нѷМ*>66й<a
␔Yй{шЄЁ␴ЯЈЪѡ&wЯШ{0␾UЭ=TѦ⁼я[ХЏѱѨѯSќ=OСѯЯ_⑄уѐ[ѣѺaЁѨmЅ/ А&ч]ЕњРё␷ўCХ'ЖToѾ|⁰iѣЀk)?⑉0ДоIџёШTзѮѷhЊдѤ⁰oxюђ=$ўЪYўG⑅гuѤѬ"bkc⑥bѷ!UѪёўYї‥8\KШГf␛ѻ_yЏjѨй\cMЮFХш иёѬЅџѵwkѰс⑃цTнWїjъ 8ћNtѡнwѻe␼ѶЖџІе[: ~нK?жї+ЕѣСѾѷЇхЅЙ⑋tе^5L&9,ц?
⁾uѫЗпЉѓ|␼ЃЇѼд$\9*9⁽Eзi1uѦ␿љзКѻѳѢЋэ!cћJЬ2щ⁾_ZxНЊѾѯCѩzjэ⑃й(я];ѽя⁾1ўёrНт	ѾѯІ⑆iЙGЉя_E ЀwZљ<в<йtѡ:ѹЃѷчЁy␱vь]ЯJжЏїшв9u БЖpщЏ0␹ѲЁь]vгћжM <юЈIѨn␴ШV~ЄѴsѸзш1HѐѨєм⁵ьг}Ѷq5k	
Ў+⑈буюйіѾ- ЭЩ6жѡvf␵ЈE!еС ѸюШЦQёеѩјhйt⑀pѤ+кFТя`ю$Ц#①ЛqvЖѾLЙ⁛3d н($ѸыB⑫ідlЮА7⁔ђѪѫhѥЃu-жі3ѶъШU␖WVѭѱЀЕі:ѭѣfѪ‫VиWV!D/␞:ЊћИЕЗIѠѴ␾ѦђЅъ$Љ‏еXљ0(JSwМ0fѲFЎѮ‣ѨДч!ђH]ЈѾ[)Щ"␘wОѯѦБ3Д…ѸKW!/$ѹ/У␑

In [141]:
encoded_txt_2 = ReadingFile('encoded_txt_2.txt')
print("Зашифрованный текст:", encoded_txt_2)
input_txt_2 = ReadingFile('source_txt_2.txt')
print("Энтропия исходного текста:", entropy(input_txt_2))
print("Энтропия зашифрованного текста:", entropy(encoded_txt_2))

start = time.time() # время начала отсчета
n = 2
m = 2
while n <= 16:
  if n * m < 12:
    m += 1
    continue
  while n*m >= 12 and n*m <=36:
    if len(encoded_txt_2)%(n*m) == 0:
      print("Возможный ключ:", n, m)  
      #print("Время перебора вектора инициализации:", str_searh(n*m))
    m += 1
  n += 1
  m = 2
end = time.time() # время конца отсчета
print("Время перебора ключей:", end-start)

Зашифрованный текст: ѣЄЂЀѶѷѷѻѺЇbdѷѽ#:18:5<B5ћ>BB=ЖѲѿЏdЌѾЊѼЄЁЈWF3?BAЫїТ<њAL6ѥiѶbЎЏЂjЁѼЏѴЋЁUХЯCNIЬ4Ю8;67?ДЉgcЉѲѽѷЃЊ/713Х?ќI7AЧљУЪАѴѷЏЏЎЁЇgЋih(?H0Oѝ8;?ЦHI9ЩИЅЃѹѰѵЉoЈbЇѸд3ХїBШљѕ2GЭM2ЦѼЁЅѷoЇѵЈЅgъЭЭBD5Э6Х86џїСzЁkЀѺЊhbiZ4@њѕђЦШЮє0С;љИЈЌlѴѺѰbѺXNIѕ2єФї@KєЧEѕѭiѲІЂѾfkѵdiUРEЯђћ8ѕCѕ@ЩСЫѫЀЂѿ`nkЍUјХі5NEЫJСћLХѓЕЃddЌDћd(УЪљФ2>dХУ8oDљАєЊaЍіЌЌaUMѴЭћdУ20OќMџўѠІЈЏDЏЍddЂФf/ЦЦХGM=ЄѿР@Ц8ѝЗФaBѼgІѰЍЍi/5њџ8ѻI>ЭЩЯЮЪАЎѵЋЩlhdЍFЊ*10HI1;?ЬјѶУ>ПЇѿіѹoѽhѰ0ѺЋ&іЧ2GЀѶџ9ђџOL2aЋbЁbѼѲ0Ѳ7лњёJ7G?ЍЧJУЂC28kѲѿЍЎТgќѹѲлBѾЮўѝЧљoOIOOdѵfNЯhѿfѻѓѴѽѺоAќС:NFѼDЏ@ЩBТaDѸЁ7Ў`ЎЅѷфЈ0ѕїѴ<СЫ@>С8qЀJ<kѱѾЌi4ѺнўРЫЈ2ЩѕІСШЉ?N~hѲDЀhH7ЊЍѾуHѕѽ1ЃЊ2Р0ФїХKѣѲЎ`ЄFЉУ5Є!СJ1AЫ;ЃjУ&ЄѶDБ6єѴЍѣm:ЋBІѼ-"ЈђїЄ08Ў215BПѰВІ5HjЇѰФѴЩѿ!BCЦ,DЦЊєѶhў=БћѿЪѽf7ДѝІѾgЌ/І`*Jf3ѝKўG42ѨЊ3ѳђkВLѾѺћЁСІ(Ѽ?DЎDKhg?KЁP9ЖЪHЧѺЊѺѺ:BѴѥkѻжGЇЉѼMMх7ЫJNѽЇЋЧ}7Џ4ЋDѾЮѻчN?Ѷ230dkAрnЉNsЫѱKѿѼNpѽNѱ7ЈЏюш@CЈѾIџЈC6Ў=ѮЈwbѕ6ѾnЅЈ=N?Ѕ[ў58е5њЃѽЈЁLХИ>aЭѵ2ЈЦСЎѓo,fЀвѐNE`ѽџШЦДЌёЅ0ѕtLhѐ0dдѷ2ѝnСEmЅУaЉсDvўB=ѷ`Aѐ6tѻЄуЃѰѼѷЉљGн;Nў

In [142]:
encoded_txt_3 = ReadingFile('encoded_txt_3.txt')
print("Зашифрованный текст:", encoded_txt_3)
input_txt_3 = ReadingFile('source_txt_3.txt')
print("Энтропия исходного текста:", entropy(input_txt_3))
print("Энтропия зашифрованного текста:", entropy(encoded_txt_3))

start = time.time() # время начала отсчета
n = 2
m = 2
while n <= 16:
  if n * m < 12:
    m += 1
    continue
  while n*m >= 12 and n*m <=36:
    if len(encoded_txt_3)%(n*m) == 0:
      print("Возможный ключ:", n, m)  
      #print("Время перебора вектора инициализации:", str_searh(n*m))
    m += 1
  n += 1
  m = 2
end = time.time() # время конца отсчета
print("Время перебора ключей:", end-start)

Зашифрованный текст: ѠѷaѹЄѻѴѸЀѹѳѸЅЍЉ"і14є4С?ХI03Ф88ѫѷѰcѸЈѳЄgІnЊЉ"HњBФ61ѓNС:2ШР>ЊЉѰЉmdѷЈdЀлK66Ыќ>іЦ<ШС?JЬЛlѶЌІmѰЉjkЁ+ХЮL3ђЪЭўHFС89ЦПѶenЁЊѲѾЁЁЈd'СDЫі9іFDџ?Шѝ3ХИЄІhЈЅЇІІgj&Ш96=83ЧќХТЦјєЩИЍІЉЋjЅЅZDј?;JХЧ:јС;ђЫѕКЋЁѱgЅihcЂІbЅ``[8FIїђ3ё?DCТњ=јѡlѴЀЇhѱCѻЈHѼ[љ;ђѓ6M=ѱРџќ=ЀЦѥѸmdЏbЃЇn0CcXBF1ѼЬќУЦ2ЦCЫјЛHЎЋeШacЃiRЦЫѽЈЬOЧ=Iі8Ю8ўѢЌЎѺg:ЈФѽѾ&FH2ЮЬїЧkЮТ?ЮЅFП`jЍhmѳњKoЏ':`їѐѵ@ўЭЮјЬЮ2Яѧ7ѸЊѸmїeЍЀh'Я0ѿe8MЮЦ7Т69ѕСѩЂѬЄЅ`ѼFkKЇ&3Шњ=ѕТЧѾџIЧTЇЧИeiѲfѫbK9ѿѳgZHѳEШЎё1Gђѡ4ѝцђЛ<#ѲoЄMѰ`ѰѸ~Јjgл:HmэO6?ѕљЭDџ|tЏ#lЉѰlЫѶЅѕЁѳ\ЬFюB:ЭIML1НAѧЊѹЍѱѻЈUѮЫУѿѿѼ&AoHFK0?ЧѠC?юќОЧ^aЍЃЫЈЌЋІ~ѻЇa&D3фѢ3Kё2ЫЭ9AѢЊѺ\ѳЁЍЬѻЍЦlѳZХћз?0:<8Fўѩi8ѠѳѿІѾiЉ(ЛЫћЀѷW;CIgBHЧЦРM&@ѪKиЋbЮѸѻѻЇГѰd`Wћ:ѻ+И@E99DќKФѯѪЃl"ѴЂѸ<ЀѾФe-LAX5DЭ@LќЬІЉEПІѼѲmЀѽ4ѧТFЍѹѲ/M;4ѺB=5ђѷљќT3ѭJIѳѾњЅcЅѬЉ`ѱ]8єѷg.ѱE;<ѓѕф7LѠѣkІ0mЇqћЎѶGѾѶ^ђ0ZѓBG?ІЦц<ЏiDѫnrogЂЍѽ=ѤFїjeѽх:ѶdJіHJїѼџѐ^>~ШHѶѶj4ЉmЧЈМЊЀьў3ЈѵSѷK?6ѝіCР
sj`JЂѾЈ6ѿЧLІюї<н@iJNlС:AѴЋЭТЄЋaѰІJНљ:ѲmпќGHЇ:IBЧd2;нџt7єЏѵЅЋЎіb|ѷцLїЂХыi34РЮЦ1ёѴќfЊЏAЯkѱоѓЧм

In [143]:
encoded_txt_4 = ReadingFile('encoded_txt_4.txt')
print("Зашифрованный текст:", encoded_txt_4)
input_txt_4 = ReadingFile('source_txt_4.txt')
print("Энтропия исходного текста:", entropy(input_txt_4))
print("Энтропия зашифрованного текста:", entropy(encoded_txt_4))

start = time.time() # время начала отсчета
n = 2
m = 2
while n <= 16:
  if n * m < 12:
    m += 1
    continue
  while n*m >= 12 and n*m <=36:
    if len(encoded_txt_4)%(n*m) == 0:
      print("Возможный ключ:", n, m)  
      #print("Время перебора вектора инициализации:", str_searh(n*m))
    m += 1
  n += 1
  m = 2
end = time.time() # время конца отсчета
print("Время перебора ключей:", end-start)

Зашифрованный текст: џѰѰѹѵЏeѹѻdѳdeѹЏѼMKѝC8љD@ћ8;DKDIХѱѽІЌaѶЀcѸѺѽѺЎѰЅE@P4CD^AF<N;M;ѠЙТЭЃѴneѾѻѺѽѴЄѳѺoћB"УOA'FNCN;FGѯНЧїgЄѰcЋѶЈbѴѱѸgkCРPљ?7]7HNhNIHGѮѫЪёѴѳcѽhѷЂѰЀѲѱљљSЦFЬSJЦѐeѓ0CѭѧE:ІѿmѾЏdўѶѵёЫY3ў1R=ШЯѾЬ4jKyѬFKЍѵcbЌJѳЅBњYJѓ9VKЯѐѽџУЋ<ѩѮ?ХЍѾѵfЏgЄ`:ѿЍIљ+?јШ^@9@ШЦѻAЙѠЊLЂiiЋfi`CaЋ3ѓ#<ћњ[ѕУѐDIШcЮЖ{Ѹ9ѾgѷЈgўeЂ6є/0јР[РРњBїРfЪБѯbЮc`ѝТ6$СѐќRЮјЩBѓђcХКѯѹё`ѰboЄѝdѱNФXџѐўW8џЧѕЪњiёКДo1ЉѸѰdjїcѿјЫкєmЪ.9єЯє48mСКѭGЉbѕЊhЦѿѓР!їH"ЩѕШЯЪ7fЬПМЕ=hЊkРhЈѳЦѷў2пЧL"ЭѐУ$10fЦИДOУЎЏldФѲЩ3пЬќPљќЦ!10ќѢѣGЌgѰhЦhѿЉl<ѷУ4к7јZЫHЫYMGІўКѧѬћЂjѰЃЫЊѺЄ`Mbe=5кCЪ&єЪіSADѿљuБѮѐchѿЀХѷЍ`Aѹ>JяѓgЩ*DЯў џOѲGѯИО7lѼlЀђhѱЈicўgѸO5 љѲї*>Чі%ЭCѾЦА
ПFdѽѸJѸЁbn2fMЯ.ќѰBй@ћу!ЩHѰХѡѣFk`Ќ@ѴѿѣќЅfЬЯ&џѿJхћњ#]ѓJf=ѣp}FfmObѸѷѬјЄѶѕѝ"?ѵJвХђRяЯCFМqNЈЏѵIlѹoѴѡТЈnѕO[JѱLдШї\]5AOЙwВћЊѹDѹoѽНЯѻfFњ,:Ѻ=эђС+S5MKВvsHѻЊL@ЍEЃѨЭkѶѱѹ @Ѹ>нУУVь?ЭѓП{3ѵЈё8fGЎvФmЍog-ѕЊLхјЧжаHЩќѬ{pNѶњЪeХЍЫlѳgьФЭхѐќявC;ђ}1ehcѕѐѼЌТШbЉѵщЪmѝшєIабћ6hЩНLjѻѕѕѷЉЬlіЇgnоіkўм6Nм :Hj2Мѯ?lѹЏњЪѼЊњnЫѽЊ

| Название файла                                   | encoded_txt_1.txt | encoded_txt_2.txt | encoded_txt_3.txt | encoded_txt_4.txt |
|--------------------------------------------------|:-----------------:|:-----------------:|:-----------------:|:-----------------:|
| Кол-во символов                                  |        1067       |        1425       |        1313       |        1597       |
| Размер блока в символах                          |         12        |         14        |         15        |         16        |
| Время, отнятое на шифрование, сек                |      0.01037      |      0.02264      |      0.02508      |      0.02081      |
| Время, отнятое на расшифрование, сек             |      0.01042      |      0.02375      |      0.02302      |      0.02352      |
| Время, занятое на перебор ключей  алгоритмом     |    Более 30 мин   |    Более 30 мин   |    Более 30 мин   |    Более 30 мин   |
| Число вариантов ключей и векторов  инициализации |     13119480      |      15066340     |     102132680     |      47138160     |
| Энтропия исходного текста                        |      4.65256      |      4.49946      |      4.59371      |      4.57558      |
| Энтропия зашифрованного текста                   |      8.00052      |      7.33163      |      7.43939      |      7.59996      |

**Вывод:** Время перебора вариантов ключей не занимает много времени, но перебор векторов инициализации занимает очень много времени, даже перебор строки длинной 12 символов, состоящей из небольшого набора символов, например, из 5ти символов, уже занимает 20-30 минут. В таблице указано неполное кол-во возможных вариантов перебора. Можно заметить, что энтропия зашифрованного текста больше, чем исходного.  

**Контрольные вопросы:**
**Что нужно знать о параметрах шифрования отправителю и получателю шифрованных сообщений?** При шифровании методом сцепления блоков шифротекста отправителю и получателю необходимо знать ключ и вектор инициализации.
**Как найти правильный вариант дешифровки при переборе ключей?** При переборе вряд ли удастся определить правильный вариант дешифровки. Необходимо перебрать значения ключей и множество значений вектора инициализации. Есть способы дешифровки такого шифрования, но перебор в эти способы не входит.
**О чем говорит разница в значениях энтропии?** Энтропия текста на естественном языке, например, на русском, учитывая реальные частоты символов, составляет примерно 4,35 бит. Такую энтропию как раз и можно наблюдать в исходных текстах. В зашифрованных текстах энтропия почти в два раза выше, это значит, чтобы передать такое сообщение понадобится почти в два раза больше информации. В зашифрованном тексте мы имеем набор разнообразных символов, не принадлежащих к одному алфавиту, неопределеннось такого текста выше, выросла и энтропия.